In [45]:
import numpy as np
from pathlib import Path
import json, subprocess, re
from typing import List, Dict, Iterator, Pattern
import pandas as pd
from pandas import DataFrame as df

In [46]:
def lambda_to_period(l: float):
    return 1 / (l / 86400)

msgs_per_day = [3, 10, 50, 100, 400]

periods = list(map(lambda_to_period, msgs_per_day))
print(periods)

[28800.0, 8640.0, 1728.0, 864.0, 216.0]


In [47]:
MAIN_SCRIPT = "./run_main.sh"
CONFIG_FILE = "main.config"

def run_from_config(config_path: str = "config.json", runner: str = "./run_main.sh", dry_run: bool = False) -> None:
    base_dir = Path.cwd()
    cfg_path = base_dir / config_path
    if not cfg_path.exists():
        raise FileNotFoundError(f"Config file not found: {cfg_path}")
    try:
        cfg = json.loads(cfg_path.read_text(encoding="utf-8"))
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON in {cfg_path}: {e}") from e

    desert_env = cfg.get("desert_env")
    rounds = cfg.get("rng_rounds", 1)
    start_rng = cfg.get("rng_start", 1)
    comb: Dict[str, List] = cfg.get("combine", {})
    iterations: List[Dict[str, Any]] = cfg.get("iterations", [])

    print(f"Loaded: rng_rounds={rounds}, rng_start={start_rng}, entries={len(iterations)}")

    cmds: List[str] = []
    if desert_env:
        env_file = (base_dir / desert_env).resolve()
        if not env_file.exists():
            raise FileNotFoundError(f"desert_env file not found: {env_file}")
        # Source environment first
        cmds.append(f"source '{env_file}'")

    if "nn" not in comb:
        comb["nn"] = [0]

    if "period" not in comb:
        comb["period"] = [0]

    if "sink_mode" not in comb:
        comb["sink_mode"] = [0]

    for nn_param in comb["nn"]:
        for period_param in comb["period"]:
            for sink_mode_param in comb["sink_mode"]:
                entry = {
                    "nn": nn_param,
                    "period": period_param,
                    "sink_mode": sink_mode_param
                }
                iterations.append(entry)


    for i, it in enumerate(iterations):
        if not isinstance(it, dict):
            raise ValueError(f"iterations[{i}] must be an object, got {type(it).__name__}")
        nn = it.get("nn")
        period = it.get("period")
        sink_mode = it.get("sink_mode")
        cmds.append(f"{runner} {nn} {period} {sink_mode} {rounds} {start_rng}")

    sep = " && "
    full_cmd = sep.join(cmds)

    if dry_run:
        print("DRY_RUN chained command:\n", full_cmd)
        return

    print("Executing bash command...")
    print(full_cmd)
    completed = subprocess.run(["bash", "-c", full_cmd], cwd=base_dir)
    if completed.returncode != 0:
        raise RuntimeError(f"Chained command failed with exit code {completed.returncode}")
    print("Success!")

run_from_config(dry_run=False)

Loaded: rng_rounds=20, rng_start=1, entries=0
Executing bash command...
source '/home/dmytro/projects/DESERT_Underwater/DESERT_buildCopy_LOCAL/environment' && ./run_main.sh 1 28800.0 1 20 1 && ./run_main.sh 1 28800.0 3 20 1 && ./run_main.sh 1 8640.0 1 20 1 && ./run_main.sh 1 8640.0 3 20 1 && ./run_main.sh 1 1728.0 1 20 1 && ./run_main.sh 1 1728.0 3 20 1 && ./run_main.sh 1 864.0 1 20 1 && ./run_main.sh 1 864.0 3 20 1 && ./run_main.sh 1 216.0 1 20 1 && ./run_main.sh 1 216.0 3 20 1 && ./run_main.sh 10 28800.0 1 20 1 && ./run_main.sh 10 28800.0 3 20 1 && ./run_main.sh 10 8640.0 1 20 1 && ./run_main.sh 10 8640.0 3 20 1 && ./run_main.sh 10 1728.0 1 20 1 && ./run_main.sh 10 1728.0 3 20 1 && ./run_main.sh 10 864.0 1 20 1 && ./run_main.sh 10 864.0 3 20 1 && ./run_main.sh 10 216.0 1 20 1 && ./run_main.sh 10 216.0 3 20 1 && ./run_main.sh 20 28800.0 1 20 1 && ./run_main.sh 20 28800.0 3 20 1 && ./run_main.sh 20 8640.0 1 20 1 && ./run_main.sh 20 8640.0 3 20 1 && ./run_main.sh 20 1728.0 1 20 1 && ./r

KeyboardInterrupt: 

### Reading trace data

In [41]:
# nn, sink_mode,
#

In [42]:
sink_id_dict = {
    "0": 254,
    "1": 253,
    "2": 252
}

def parse_file_records(regex: Pattern, path: str) -> Iterator[Dict]:
    """Yield one dict per matching log line from `path`."""
    with open(path, "r") as f:
        for line in f:
            m = regex.match(line)
            if not m:
                continue

            sink_id = sink_id_dict[m.group("sink_id")] if m.group("sink_id") in sink_id_dict else m.group("sink_id")
            yield {
                "time": float(m.group("time")),
                "sink_id": int(sink_id),
                "packet_id": int(m.group("packet_id")),
                "src_ip": int(m.group("src_ip")),
            }

In [43]:
DATA_PATH = "data/"

LOG_PATH = "log_udp.out"
RESULT_PATH = "output.out"

_pattern = re.compile(r"\[(?P<time>\d+(?:\.\d+)?)\]::DBG::UWUDP\((?P<sink_id>\d+)\)::recv\(Packet \*, int\)::new packet with id (?P<packet_id>\d+) from ip (?P<src_ip>\d+) : \d+")

udp_df = df.from_records(parse_file_records(_pattern, LOG_PATH))
result_df = pd.read_csv(RESULT_PATH, skiprows=17, header=0)
result_df = result_df.replace("_", np.nan).infer_objects(copy=False)
# print("Parsed rows:", len(udp_df))
print(udp_df.head())
print(result_df.head())

if udp_df.empty:
    print("No matching UDP recv lines found.")
    raise Exception()

summary = udp_df.groupby("packet_id") \
    .agg(first_time=("time", "min"), last_time=("time", "max"), duplicates=("packet_id", "count"), sinks=("sink_id", lambda s: sorted(set(s)))) \
    .reset_index()

print(summary.head())

total_recv = len(summary)
total_sent = result_df["n_pkts"].sum()
print("Total received packets:", total_recv)
print("Total transmitted packets: ", total_sent)
print(f"Total PDR: {total_recv / total_sent * 100:.2f}")


FileNotFoundError: [Errno 2] No such file or directory: 'log_udp.out'